# Zadanie 10

Zaimplementować:
* klasę `CVSplitter`,
* funkcję `train_on_best_hyperparams`,
* funkcję `double_split_evaluate`.

### `CVSplitter`

Klasa `CVSplitter` ma działać podobnie do klasy `RandomSplitter`. Metoda `get_splits` przyjmuje instancję klasy `Dataset` oraz `seed` (używany przy losowaniu splitów) i zwraca listę (długości `self.n_splits`) instancji klas `Split`. Różnica pomiędzy `CVSplitter` a `RandomSplitter` jest taka, że `CVSplitter` implementuje cross validation, tzn. każdy element datasetu znajduje się w __dokładnie jednym__ zbiorze testowym oraz rozmiary poszczególnych zbiorów testowych mogą się różnić maksymalnie o 1 (gdy rozmiar datasetu jest niepodzielny przez liczbę splitów).

### `train_on_best_hyperparams`

Funkcja `train_on_best_hyperparams` ma za zadanie dobrać najlepszy zestaw hiperparametrów i wytrenować model.

Parametry:
* `dataset` - instancja klasy `Dataset`, na której będziemy się uczyć,
* `model_cls` - klasa modelu, który chcemy wytrenować; w `__init__` przyjmuje `X`, `y` oraz pozostałe hiperparametry; ponadto ma metodę `predict`, która standardowo przyjmuje tablicę typu `X` i zwraca tablicę typu `y`,
* `hyperparams_list` - lista zestawów hiperparametrów, z których chcemy wybrać najlepszy; każdy element `h` listy `hyperparams_list` jest słownikiem; uczenie modelu na danym zestawie hiperparametrów wygląda następująco: `model = model_cls(dataset.X, dataset.y, **h)`,
* `splitter` - instancja klasy dziedziczącej po `Splitter`,
* `score_function` - funkcja przyjmuje jako parametry `y_true` oraz `y_pred`, zwraca wynik (float) - im większy, tym lepiej,
* `seed` - podawane do splittera.

Funkcja zwraca:
* `model` - model wytrenowany na __całym datasecie__ przy użyciu znalezionych najlepszych hiperparametrów,
* `best_h` - (słownik) najlepszy zestaw hiperparametrów,
* `train_scores` - tablica o wymiarach (liczba_splitów, liczba_zestawów_hiperparametrów), w której umieszczone są wyniki policzone na zbiorach treningowych (`split.train.X`, `split.train.y`),
* `test_scores` - tablica o wymiarach (liczba_splitów, liczba_zestawów_hiperparametrów), w której umieszczone są wyniki policzone na zbiorach testowych (`split.test.X`, `split.test.y`).

Funkcja działa w następujący sposób:
* w podwójnej pętli:
 * iterujemy po wszystkich splitach (splitter + dataset),
 * iterujemy po wszystkich zestawach hiperparametrów,
 * trenujemy model, obliczamy i zapisujemy jego score na zbiorze treningowym i testowym
* dla każdego zestawu hiperparametrów uśredniamy score'y po wszystkich splitach, wybieramy najlepszy zestaw hiperparametrów (mający najwyższy średni score),
* trenujemy model na całym datasecie z użyciem najlepszych hiperparametrów,
* zwracamy wyniki.

### `double_split_evaluate`

Funkcja `double_split_evaluate` ma za zadanie wyestymować score modelu uczonego funkcją `train_on_best_hyperparams`. W połączeniu ze splitterem `CVSplit` implementuje tzw. _double cross validation_.

Parametry:
* `dataset` - instancja klasy `Dataset`, na której będziemy się uczyć,
* `model_cls` - klasa modelu, który chcemy wytrenować; w `__init__` przyjmuje `X`, `y` oraz pozostałe hiperparametry; ponadto ma metodę `predict`, która standardowo przyjmuje tablicę typu `X` i zwraca tablicę typu `y`,
* `hyperparams_list` - lista zestawów hiperparametrów, z których chcemy wybrać najlepszy; każdy element `h` listy `hyperparams_list` jest słownikiem; uczenie modelu na danym zestawie hiperparametrów wygląda następująco: `model = model_cls(dataset.X, dataset.y, **h)`,
* `major_splitter` - instancja klasy dziedziczącej po `Splitter`,
* `minor_splitter` - instancja klasy dziedziczącej po `Splitter`,
* `score_function` - funkcja przyjmuje jako parametry `y_true` oraz `y_pred`, zwraca wynik (float) - im większy, tym lepiej,
* `seed` - podawane do splitterów.

Funkcja zwraca słownik `summary`, który zawiera:
* "best_hyperparams" - lista najlepszych zestawów hiperparametów na każdy major split (długości liczba_major_splitów),
* "train_scores" - tablica o długości liczba_major_splitów, w której umieszczone są wyniki policzone na zbiorach treningowych (`major_split.train.X`, `major_split.train.y`) z użyciem znalezionych najlepszych zestawów hiperparametrów,
* "test_scores" - tablica o długości liczba_major_splitów, w której umieszczone są wyniki policzone na zbiorach testowych (`major_split.test.X`, `major_split.test.y`) z użyciem znalezionych najlepszych zestawów hiperparametrów,
* "estimated_score" - średnia "test_scores", główny wynik działania tej funkcji,
* "inner_train_scores": tablica o wymiarach (liczba_major_splitów, liczba_minor_splitów, liczba_zestawów_hiperparametrów), w której zebrane są `train_scores` zwracane przez `train_on_best_hyperparams`,
* "inner_test_scores": tablica o wymiarach (liczba_major_splitów, liczba_minor_splitów, liczba_zestawów_hiperparametrów), w której zebrane są `test_scores` zwracane przez `train_on_best_hyperparams`,

Funkcja działa w następujący sposób:
* iterujemy po wszystkich major splitach przy użyciu `major_splitter`,
 * z każdego major splitu wyciągamy dataset treningowy i podajemy go do funkcji `train_on_best_hyperparams` razem z `minor_splitter` i pozostałymi parametrami, zbieramy wyniki,
 * testujemy zwrócony model na major splicie, dodajemy wyniki do `summary["train_scores"]` oraz `summary["test_scores"]`,
* obliczamy `summary["estimated_score"]`,
* zwracamy `summary`.

### Uwagi
* Funkcja `train_on_best_hyperparams` sprawia, że dobór hiperparametrów staje się częścią algorytmu uczenia modelu (czyli hiperparametry stają się parametrami).
* Tzw. "ręczny dobór hiperparametrów" jest oszustwem i należy zawsze go unikać.
* Istnieją lepsze metody szukania najlepszych hiperparametrów, np. uczenie bayesowskie.
* Skoro wewnętrzna cross validacja (minor splits) stała się częścią algorytmu uczenia, to nie powinno się używać jej wyników do estymacji score na nowych danych - stąd konieczność dodania zewnętrznej cross validacji (major splits).
* Ostateczny model zawsze powinien być uczony na pełnym datasecie (im więcej danych, tym lepiej), niezależnie od tego, jak wyglądał algorytm znajdywania hiperparametrów oraz ewaluacji.
* Podwójnej cross validacji używamy zazwyczaj wtedy, gdy mamy mało danych - w wypadku dużych datasetów może wystarczyć pojedynczy train/valid/test split. Można też stosować podejście mieszane, tzn. użyć `RandomSplitter` albo zmodyfikować `CVSplitter` tak, żeby zwracał tylko część policzonych splitów.
* Pamiętamy, że pruning drzew decyzyjnych wymagał podzielenia danych na dwie części. Jeśli dodamy do tego podwójną cross validację, to otrzymamy coś na kształt "potrójnej cross validacji" - nie ma w tym nic "matematycznie złego", przy czym im bardziej zagnieżdżoną cross validację chcielibyśmy stosować, tym mniejszy dataset treningowy dostaniemy na samym końcu i tym dłużej potrwa całe uczenie.
* Należy bardzo uważać, jaki splitter stosujemy! Jeśli dane nie są I.I.D., to nie możemy użyć po prostu `RandomSplitter` lub `CVSplitter` (dlaczego?).